In [1]:
file_path=(r"C:\Users\WIIS\Downloads\SYSTEM SOFTWARE-Ktunotes.in.pdf")

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Extract text from documents
raw_texts = [doc.page_content for doc in pages]

# Split text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents(raw_texts)

In [ ]:
#initializing the embedding model
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

d:\PDF_Query_Chatbot\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\PDF_Query_Chatbot\chatbot\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\WIIS\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate d

In [ ]:
#Embedding and creating the vector store
from langchain_chroma import Chroma

db = Chroma.from_documents(texts,embeddings_model)

In [30]:
retriever = db.as_retriever(search_kwargs={"filter":{"id":"1"}})

In [ ]:
# Initialize the Hugging Face model
from langchain_huggingface import ChatHuggingFace
from langchain_community.chat_models import ChatHuggingFace as CommunityChatHuggingFace
from huggingface_hub import InferenceClient
from langchain_huggingface.llms import HuggingFaceEndpoint

HF_TOKEN="Your HF Token"


# Create endpoint LLM correctly
llm_endpoint = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token=HF_TOKEN,
    task="text-generation",
    temperature=0.3,         
    max_new_tokens=1024       

# Wrap into chat model
llm = ChatHuggingFace(llm=llm_endpoint)


In [37]:
# Define the prompt template
template = """
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create the prompt and output parser
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# Define the chain (pipeline)
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [39]:
# Test the chain with a query
print(chain.invoke("list the course outcomes"))


context: A course in [topic] aims to provide learners with the following outcomes:

1. Understanding of the basic concepts and principles of [topic]
2. Familiarity with key terminologies and definitions related to [topic]
3. Proficiency in using [specific software/tools] for [specific tasks/functions]
4. Ability to apply [topic] concepts and principles to real-world situations
5. Knowledge of best practices and industry standards in [topic]
6. Skills to analyze and solve problems related to [topic]
7. Development of [specific skills/competencies] related to [topic]
8. Familiarity with current trends and developments in [topic]
9. Ability to critically evaluate and interpret [topic] related data and information
10. Capacity to communicate effectively about [topic] with stakeholders.

Course outcomes are the measurable and achievable goals that learners should be able to accomplish by the end of the course. They indicate what learners will be able to do as a result of taking the course.